## Item Demand Forecasting

**Importing Libraries**

In [83]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import lightgbm as lgb
import warnings


pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
warnings.filterwarnings('ignore')

In [84]:
df = pd.read_csv("train 2.csv")

In [85]:
def check_df(dataframe, head=5, tail=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Data Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(tail))
    print("##################### Null VAlues #####################")
    print(dataframe.isnull().sum())

check_df(df)

##################### Shape #####################
(913000, 4)
##################### Data Types #####################
date     object
store     int64
item      int64
sales     int64
dtype: object
##################### Head #####################
         date  store  item  sales
0  2013-01-01      1     1     13
1  2013-01-02      1     1     11
2  2013-01-03      1     1     14
3  2013-01-04      1     1     13
4  2013-01-05      1     1     10
##################### Tail #####################
              date  store  item  sales
912995  2017-12-27     10    50     63
912996  2017-12-28     10    50     59
912997  2017-12-29     10    50     74
912998  2017-12-30     10    50     62
912999  2017-12-31     10    50     82
##################### Null VAlues #####################
date     0
store    0
item     0
sales    0
dtype: int64


There are no null values in this data 

We have to change the data type of feature 'date' to datetime 

In [86]:
df['date'] = pd.to_datetime(df['date'])

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 913000 entries, 0 to 912999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   date    913000 non-null  datetime64[ns]
 1   store   913000 non-null  int64         
 2   item    913000 non-null  int64         
 3   sales   913000 non-null  int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 27.9 MB


In [88]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
store,913000.0,5.500000,2.872283,1.0,3.0,5.5,8.0,10.0
item,913000.0,25.500000,14.430878,1.0,13.0,25.5,38.0,50.0
sales,913000.0,52.250287,28.801144,0.0,30.0,47.0,70.0,231.0


In [89]:
df.nunique()

date     1826
store      10
item       50
sales     213
dtype: int64

In [90]:
df['date'].min() # finding the minimum date

Timestamp('2013-01-01 00:00:00')

In [91]:
df['date'].max() # finding the maximum date

Timestamp('2017-12-31 00:00:00')

In [92]:
df['item'].value_counts()

1     18260
38    18260
28    18260
29    18260
30    18260
31    18260
32    18260
33    18260
34    18260
35    18260
36    18260
37    18260
39    18260
2     18260
40    18260
41    18260
42    18260
43    18260
44    18260
45    18260
46    18260
47    18260
48    18260
49    18260
27    18260
26    18260
25    18260
24    18260
3     18260
4     18260
5     18260
6     18260
7     18260
8     18260
9     18260
10    18260
11    18260
12    18260
13    18260
14    18260
15    18260
16    18260
17    18260
18    18260
19    18260
20    18260
21    18260
22    18260
23    18260
50    18260
Name: item, dtype: int64

In [93]:
df['date'].value_counts()

2013-01-01    500
2016-05-11    500
2016-05-09    500
2016-05-08    500
2016-05-07    500
             ... 
2014-08-29    500
2014-08-28    500
2014-08-27    500
2014-08-26    500
2017-12-31    500
Name: date, Length: 1826, dtype: int64

**Group by the data with 'date' & 'item' and find Sum of the 'Sales'**

In [94]:
data = df.copy()

In [95]:
df = df.drop(['store'] , axis = 1)

In [96]:
df = df.groupby(['date', 'item']).sum()

In [97]:
df.head(20)

sales
date       item       
2013-01-01 1       133
           2       327
           3       172
           4       102
           5        83
           6       288
           7       278
           8       416
           9       245
           10      378
           11      369
           12      369
           13      444
           14      310
           15      453
           16      144
           17      171
           18      415
           19      208
           20      265

In [98]:
df = df.reset_index()

In [99]:
df.head()

,date,item,sales
0,2013-01-01,1,133
1,2013-01-01,2,327
2,2013-01-01,3,172
3,2013-01-01,4,102
4,2013-01-01,5,83


In [100]:
def Sales(sales_df):
  temp = sales_df['date']
  sales_df['sales'] = sales_df['sales'].rolling(90).sum().shift(-90)
  sales_df['date'] = temp
  return sales_df

In [101]:
df = df.groupby(['item']).apply(lambda sales_df: Sales(sales_df))

In [102]:
df.head(10)

,date,item,sales
0,2013-01-01,1,12848.0
1,2013-01-01,2,34724.0
2,2013-01-01,3,21472.0
3,2013-01-01,4,12940.0
4,2013-01-01,5,10678.0
5,2013-01-01,6,34380.0
6,2013-01-01,7,34470.0
7,2013-01-01,8,45320.0
8,2013-01-01,9,30547.0
9,2013-01-01,10,42939.0


## Feature Engineerig

In [103]:
def create_date_features(df):
    df['month'] = df.date.dt.month
    df['week_of_year'] = df.date.dt.weekofyear
    df['day_of_week'] = df.date.dt.dayofweek
    df['year'] = df.date.dt.year
    df["is_wknd"] = df.date.dt.weekday //5
    df['is_month_start'] = df.date.dt.is_month_start.astype(int)
    df['is_month_end'] = df.date.dt.is_month_end.astype(int)
    return df

df = create_date_features(df)
df.head()

,date,item,sales,month,week_of_year,day_of_week,year,is_wknd,is_month_start,is_month_end
0,2013-01-01,1,12848.0,1,1,1,2013,0,1,0
1,2013-01-01,2,34724.0,1,1,1,2013,0,1,0
2,2013-01-01,3,21472.0,1,1,1,2013,0,1,0
3,2013-01-01,4,12940.0,1,1,1,2013,0,1,0
4,2013-01-01,5,10678.0,1,1,1,2013,0,1,0


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91300 entries, 0 to 91299
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            91300 non-null  datetime64[ns]
 1   item            91300 non-null  int64         
 2   sales           86800 non-null  float64       
 3   month           91300 non-null  int64         
 4   week_of_year    91300 non-null  int64         
 5   day_of_week     91300 non-null  int64         
 6   year            91300 non-null  int64         
 7   is_wknd         91300 non-null  int64         
 8   is_month_start  91300 non-null  int64         
 9   is_month_end    91300 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(8)
memory usage: 9.7 MB
